In [103]:
import pandas as pd
import pm_lib
import ipywidgets as widgets
import sqlite3
from IPython.display import clear_output, display
from tkinter import Tk, filedialog

In [104]:
files = ''

def select_files(b):
    global files
    clear_output()
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    files = filedialog.askopenfilename(title='Choose a File', filetypes=[("JFLAP files", "*.jff")]) 

def select_files_multiple(b):
    global files
    clear_output()
    root = Tk()
    root.withdraw() 
    root.call('wm', 'attributes', '.', '-topmost', True) 
    files = filedialog.askopenfilename(title='Choose a File', multiple=True, filetypes=[("JFLAP files", "*.jff")]) 

def select_csv_files(b):
    global files
    clear_output()
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    files = filedialog.askopenfilename(title='Choose a File', filetypes=[("CSV files", "*.csv")])

layout = widgets.Layout(width='170px')

fileselect = widgets.Button(description="Selecione o arquivo", layout=layout)
fileselect.on_click(select_files)

fileselect_multiple = widgets.Button(description="Selecione os arquivos", layout=layout)
fileselect_multiple.on_click(select_files_multiple)

fileselect_csv = widgets.Button(description="Selecione o arquivo", layout=layout)
fileselect_csv.on_click(select_csv_files)


In [105]:
col_names = ['Palavra', 'Esperado']


def on_button_clicked_nfa(b):
    global files
    results = []
    con = sqlite3.connect('./database/nfaCases.db')
    
    with out1:
        problem_sel = nfa_problem_sel.value
        query_str = f'SELECT * FROM ' + problem_sel
        res =  pd.read_sql_query(query_str, con)
        con.close()
        doc = 'nfa.csv'
        nfa = pm_lib.jffToNFA(files)
        res = res.to_csv(doc, index=False, header=False)
        data = pd.read_csv(doc, sep=',',dtype=object, names = col_names)
        g = pm_lib.nfa_to_graph(nfa)
        results = pm_lib.nfa_validate(nfa, doc)
        df = pd.DataFrame(columns=["Recebido"])
        df["Recebido"] = results
        df = pd.concat([data, df], axis=1)
        df = df.dropna()
        #df.to_csv('file_sent.csv', sep=',', encoding='utf-8', index=False)
        conn = sqlite3.connect('./database/nfaResults.db')
        df.to_sql(name=problem_sel,con=conn, if_exists='replace')
        #Closing the connection
        conn.close()
        clear_output()
        display(widgets.HTML("<h3> NFA gerado: </h3>"))
        display(g)
        display(df)
        if df['Esperado'].equals(df['Recebido']):
            sucess=widgets.Button(description="Correto!", button_style='success', layout=layout)
            sucess.disabled = True
            display(sucess)
        else:
            layout2 = widgets.Layout(width='400px')
            fail=widgets.Button(description="Erros foram encontrados, verifique a tabela de resultados.",
                button_style='danger', layout=layout2)
            fail.disabled = True
            display(fail)

def on_button_clicked_dfa(b):
    global files
    results = []
    con = sqlite3.connect('./database/dfaCases.db')
    
    with out2:
        problem_sel = dfa_problem_sel.value
        query_str = f'SELECT * FROM ' + problem_sel
        res =  pd.read_sql_query(query_str, con)
        con.close()
        doc = 'dfa.csv'
        dfa = pm_lib.jffToDFA(files)
        res = res.to_csv(doc, index=False, header=False)
        data = pd.read_csv(doc, sep=',',dtype=object, names = col_names)
        g = pm_lib.dfa_to_graph(dfa)
        results = pm_lib.dfa_validate(dfa, doc)
        df = pd.DataFrame(columns=["Recebido"])
        df["Recebido"] = results
        df = pd.concat([data, df], axis=1)
        df = df.dropna()
        # df.to_csv('file_sent.csv', sep=',', encoding='utf-8', index=False)
        conn = sqlite3.connect('./database/dfaResults.db')
        df.to_sql(name=problem_sel,con=conn, if_exists='replace')
        #Closing the connection
        conn.close()
        clear_output()
        display(widgets.HTML("<h3> DFA gerado: </h3>"))
        display(g)
        display(df)
        if df['Esperado'].equals(df['Recebido']):
            sucess=widgets.Button(description="Correto!", button_style='success', layout=layout)
            sucess.disabled = True
            display(sucess)
        else:
            layout2 = widgets.Layout(width='400px')
            fail=widgets.Button(description="Erros foram encontrados, verifique a tabela de resultados.",
                button_style='danger', layout=layout2)
            fail.disabled = True
            display(fail)



In [106]:
def on_button_clicked_union(b):
    global files

    with out3:
        if select_automata_type.value == 'NFA':
            automata1 = pm_lib.jffToNFA(files[0])
            automata2 = pm_lib.jffToNFA(files[1])
            united_automata = pm_lib.automata_nfa_union(automata1, automata2)
            g = pm_lib.nfa_to_graph(united_automata)
            clear_output()
            display(g)
        else:
            automata1 = pm_lib.jffToDFA(files[0])
            automata2 = pm_lib.jffToDFA(files[1])
            united_automata = pm_lib.automata_dfa_union(automata1, automata2)
            g = pm_lib.dfa_to_graph(united_automata)
            clear_output()
            display(g)

def on_button_clicked_intersection(b):
    global files

    with out4:
        if select_automata_type.value == 'NFA':
            automata1 = pm_lib.jffToNFA(files[0])
            automata2 = pm_lib.jffToNFA(files[1])
            intersection_automata = pm_lib.automata_nfa_intersection(automata1, automata2)
            g = pm_lib.nfa_to_graph(intersection_automata)
            clear_output()
            display(g)
        else:
            automata1 = pm_lib.jffToDFA(files[0])
            automata2 = pm_lib.jffToDFA(files[1])
            intersection_automata = pm_lib.automata_dfa_intersection(automata1, automata2)
            g = pm_lib.dfa_to_graph(intersection_automata)
            clear_output()
            display(g)


In [107]:
def on_button_clicked_send_csv(b):
    global files
    with out6:
        if select_automata_type.value == 'DFA':
            conn = sqlite3.connect('./database/dfaCases.db')
            df = pd.read_csv(files, header=None, index_col=False)
            df.to_sql(name=text_area.value,con=conn, if_exists='replace', index=False)
            
            conn.close()
            # clear_output()
            # display(print('Caso de teste cadastrado com sucesso!'))
            conn_dfa = sqlite3.connect('./database/dfaCases.db')
            dfa_problem_sel.options = tables_in_sqlite_db(conn_dfa)

        else:
            conn = sqlite3.connect('./database/nfaCases.db')
            df = pd.read_csv(files, header=None, index_col=False)
            df.to_sql(name=text_area.value,con=conn, if_exists='replace', index=False)
            conn.close()
            # clear_output()
            # display(print('Caso de teste cadastrado com sucesso!'))

            conn_nfa = sqlite3.connect('./database/nfaCases.db')
            nfa_problem_sel.options = tables_in_sqlite_db(conn_nfa)

In [108]:
out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()
out5 = widgets.Output()
out6 = widgets.Output()

button_nfa = widgets.Button(description='Avaliar', layout = layout)
button_dfa = widgets.Button(description='Avaliar', layout = layout)
button_union = widgets.Button(description='União', layout = layout)
button_intersection = widgets.Button(description='Interseção', layout = layout)
select_automata_type = widgets.Dropdown(options=['NFA', 'DFA'])
button_send_csv = widgets.Button(description='Cadastrar', layout=layout)

button_nfa.on_click(on_button_clicked_nfa)
button_dfa.on_click(on_button_clicked_dfa)
button_union.on_click(on_button_clicked_union)
button_intersection.on_click(on_button_clicked_intersection)
button_send_csv.on_click(on_button_clicked_send_csv)

text_area = widgets.Textarea(value='')

nfa_title =  widgets.HTML(value="<h3>Autômato Finito não Determinístico</h3>")
nfa_description = widgets.HTML(value= "<p> Um autômato finito não determinístico" +
    " pode ter transições para mais de um estado ao ler como entrada " +
    "um mesmo valor. </p>" +
    "<p> Clique em avaliar para ver o resultado do NFA em relação aos valores esperados. </p>"
    )

dfa_title =  widgets.HTML(value="<h3>Autômato Finito Determinístico</h3>")
dfa_description = widgets.HTML(value="<p> Um autômato finito determinístico" +
    " não pode ter transições para mais de um estado ao ler como entrada " +
    "um mesmo valor. </p>"
    "<p> Clique em avaliar para ver o resultado do DFA em relação aos valores esperados. </p>"
    )

union_title =  widgets.HTML(value="<h3>União</h3>")
union_decription = widgets.HTML(value="<img src='images/union.png' alt='União' style='width:220px;height:132px;'>"+
    " <p> Você pode realizar a união de dois autômatos. "+
    "Selecione os dois arquivos '.jff' que deseja unir e clique em 'União'. </p>"
    )

intersection_title =  widgets.HTML(value="<h3>Interseção</h3>")
intersection_decription = widgets.HTML(value="<img src='images/intersection.png' alt='Interseção'>"+
    " <p> Você pode realizar a interseção de dois autômatos. "+
    "Selecione os dois arquivos '.jff' que deseja fazer a interseção e clique em 'Interseção'. </p>"
    )
    
questoes_title =  widgets.HTML(value="<h3>Adicionar casos de teste em databases</h3>")
string = """ <p> Adicione casos de teste a database do banco de questões. </p> 
<p> Para tal, selecione o arquivo csv que contém os casos de teste, selecione uma categoria (DFA ou NFA) e digite um nome para a database</p>
"""
questoes_decription = widgets.HTML(value=string)

def tables_in_sqlite_db(conn):
    conn.commit()
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [
        v[0] for v in cursor.fetchall()
        if v[0] != "sqlite_sequence"
    ]
    cursor.close()
    return tables


conn_dfa = sqlite3.connect('./database/dfaCases.db')
dfa_problem_sel = widgets.Dropdown(options=tables_in_sqlite_db(conn_dfa))

conn_nfa = sqlite3.connect('./database/nfaCases.db')
nfa_problem_sel = widgets.Dropdown(options=tables_in_sqlite_db(conn_nfa))


In [109]:
vbox1 = widgets.VBox([nfa_title, nfa_description, nfa_problem_sel, fileselect, button_nfa, out1])
vbox2 = widgets.VBox([dfa_title, dfa_description, dfa_problem_sel, fileselect, button_dfa, out2])
vbox3 = widgets.VBox([union_title, union_decription, fileselect_multiple, select_automata_type, button_union, out3])
vbox4 = widgets.VBox([intersection_title, intersection_decription, fileselect_multiple, select_automata_type, button_intersection, out4])
vbox5 = widgets.VBox([questoes_title, questoes_decription, fileselect_csv, select_automata_type, text_area, button_send_csv, out6])

tab = widgets.Tab(children = [vbox1, vbox2, vbox3, vbox4, vbox5])
tab.set_title(0, 'NFA')
tab.set_title(1, 'DFA')
tab.set_title(2, 'União')
tab.set_title(3, 'Interseção')
tab.set_title(4, 'Casos de teste')

display(tab)

In [110]:
# conn = sqlite3.connect('./database/dfaResults.db')

# cursor = conn.cursor()

# #Doping EMPLOYEE table if already exists.
# cursor.execute("DROP TABLE IF EXISTS q1")

# #Creating table as per requirement
# sql ='''CREATE TABLE q1(
#     palavra TEXT NOT NULL,
#     resultado TEXT NOT NULL
# );'''
# cursor.execute(sql)
# print("Table created successfully........")

# #Commit your changes in the database
# conn.commit()

# #Closing the connection
# conn.close()

# # conn = sqlite3.connect('./database/testeCases.db')
# # test_df = pd.read_sql('SELECT * FROM questoes;',conn)
# # conn.close()

# # test_df.T